In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt

2.3.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 36.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle
import json
from torch_geometric.data import Data
from torch_geometric.nn import MetaPath2Vec


In [ ]:

def json_open(file_path):
  with open(file_path, 'r') as f:
    data = json.load(f)
  return data


def pickle_open(file_path):
  with open(file_path, 'rb') as f:
    data = pickle.load(f)
  return data

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import os
os.path.exists('/content/drive/Shareddrives/SEP/Data/edges.json')
path = '/content/drive/Shareddrives/SEP/Data'

In [ ]:
edges = json_open(os.path.join(path,'edges.json'))
labels = json_open(os.path.join(path,'labels.json'))
keyword_embeddings = pickle_open(os.path.join(path,'keyword_embeddings.pkl'))
tweet_embeddings = pickle_open(os.path.join(path,'tweet_embeddings.pkl'))
user_embeddings = pickle_open(os.path.join(path,'user_embeddings.pkl'))

In [ ]:
seen = set()
for edge in edges:
  relation = edge['relation']
  if relation not in seen:
    print(relation)
    seen.add(relation)

user-follower-user
user-following-user
user-profile-keyword
user-post-tweet
user-engage-tweet
tweet-include-keyword
tweet-tag-keyword


In [ ]:
keyword_ids = keyword_embeddings['ids']
keyword_embeddings = keyword_embeddings['embeddings']

tweet_ids = tweet_embeddings['ids']
tweet_embeddings = tweet_embeddings['embeddings']

user_ids = user_embeddings['ids']
user_embeddings = user_embeddings['embeddings']


In [ ]:
print(type(edges))
for edge in edges:
  if edge['source_id'].startswith('user_') or edge['source_id'].startswith('tweet_'):
    continue
  else:
    print(edge['source_id'])


<class 'list'>


In [ ]:
from collections import defaultdict
# help(defaultdict)

edge_index_dict = defaultdict(lambda : ([], []))

for edge in edges:
  source = edge['source_id']
  target = edge['target_id']
  relation = edge['relation']
  relation = relation.split('-')
  relation = relation[1]
  relation = "to -->"
  other_relation = "<-- to"

  if source.startswith('user_') and target.startswith('user_'):
    edge_type = ('user', relation, 'user')
    edge_index_dict[edge_type][0].append(user_ids.index(source))
    edge_index_dict[edge_type][1].append(user_ids.index(target))
  elif source.startswith('user_') and target.startswith('tweet_'):
    edge_type = ('user', relation, 'tweet')
    edge_index_dict[edge_type][0].append(user_ids.index(source))
    edge_index_dict[edge_type][1].append(tweet_ids.index(target))
    edge_type = ('tweet', other_relation, 'user')
    edge_index_dict[edge_type][0].append(tweet_ids.index(target))
    edge_index_dict[edge_type][1].append(user_ids.index(source))
  elif source.startswith('user_') and target.startswith('keyword_'):
    edge_type = ('user', relation, 'keyword')
    edge_index_dict[edge_type][0].append(user_ids.index(source))
    edge_index_dict[edge_type][1].append(keyword_ids.index(target))
    edge_type = ('keyword', other_relation, 'user')
    edge_index_dict[edge_type][0].append(keyword_ids.index(targesst))
    edge_index_dict[edge_type][1].append(user_ids.index(source))
  elif source.startswith('tweet_') and target.startswith('keyword_'):
    edge_type = ('tweet', relation, 'keyword')
    edge_index_dict[edge_type][0].append(tweet_ids.index(source))
    edge_index_dict[edge_type][1].append(keyword_ids.index(target))
    edge_type = ('keyword', other_relation, 'tweet')
    edge_index_dict[edge_type][0].append(keyword_ids.index(target))
    edge_index_dict[edge_type][1].append(tweet_ids.index(source))
  else:
    raise ValueError(f"Invalid edge: {edge}")
  # edge_index_dict[edge_type][0].append(user_ids.index(source) if source.startswith('user_')
  #                                       else tweet_ids.index(source) if source.startswith('tweet_')
  #                                       else keyword_ids.index(source))
  # edge_index_dict[edge_type][1].append(user_ids.index(target) if target.startswith('user_')
  #                                       else tweet_ids.index(target) if target.startswith('tweet_')
  #                                       else keyword_ids.index(target))

edge_index_dict = {k: (torch.tensor(v[0]), torch.tensor(v[1])) for k, v in edge_index_dict.items()}


In [ ]:
edge_index_dict

{('user',
  'to -->',
  'user'): (tensor([  477, 22398, 15158,  ..., 28459, 28459, 28459]), tensor([ 2824,  2824,  2824,  ..., 18599, 11741,  6539])),
 ('user',
  'to -->',
  'keyword'): (tensor([27951, 27951, 27951,  ..., 28831, 28833, 28837]), tensor([219, 224, 173,  ...,  25,  83,  71])),
 ('keyword', '<-- to', 'user'): (tensor([219, 224, 173,  ...,  25,  83,  71]),
  tensor([27951, 27951, 27951,  ..., 28831, 28833, 28837])),
 ('user',
  'to -->',
  'tweet'): (tensor([    0,     0,     0,  ..., 28838, 28838, 28838]), tensor([    1,     7,     2,  ..., 32234, 75464, 75462])),
 ('tweet',
  '<-- to',
  'user'): (tensor([    1,     7,     2,  ..., 32234, 75464, 75462]), tensor([    0,     0,     0,  ..., 28838, 28838, 28838])),
 ('tweet',
  'to -->',
  'keyword'): (tensor([     0,      0,      0,  ..., 118696, 118696, 118696]), tensor([215,  60, 173,  ..., 175,  80, 183])),
 ('keyword', '<-- to', 'tweet'): (tensor([215,  60, 173,  ..., 175,  80, 183]),
  tensor([     0,      0,      0, 

In [ ]:
found = set()
for value in labels.values():
  if value not in found:
    print(value)
    found.add(value)

Buyer
Related
Seller
Negative


In [ ]:
from torch_geometric.data import HeteroData

data = HeteroData()

data['user'].x  = torch.tensor(user_embeddings, dtype=torch.float)
data['tweet'].x = torch.tensor(tweet_embeddings, dtype=torch.float)
data['keyword'].x = torch.tensor(keyword_embeddings, dtype=torch.float)

for edge_type, (source_id, target_id) in edge_index_dict.items():
  data[edge_type].edge_index = torch.stack([source_id, target_id], dim = 0)

y = []
y_index = []

user_id_map = {id_: i for i, id_ in enumerate(user_ids)}

for user, label in labels.items():
    # if user in user_id_map:
    #     y_index.append(user_id_map[user])
    y.append(label)


labels_map = {'Buyer' : 0, 'Related' : 1, 'Seller': 2, 'Negative': 3}
# y = torch.tensor([ labels_map[labels[user_id]]for user_id in user_ids], dtype= torch.long)
y = torch.tensor([ labels_map[label] for label in labels.values()], dtype= torch.long)

# y_index = torch.tensor(y_index)

data['user'].y = y
# data['user'].y_index = y_index

In [ ]:
print(data['user'].y)

tensor([0, 0, 0,  ..., 3, 2, 0])


In [ ]:
data['user'].y_index

tensor([    0,     1,     2,  ..., 28836, 28837, 28838])

In [ ]:
print(data.edge_index_dict)

{('user', 'follower', 'user'): tensor([[  477, 22398, 15158,  ...,  2085, 19108, 28042],
        [ 2824,  2824,  2824,  ..., 28455, 28457, 28457]]), ('user', 'following', 'user'): tensor([[ 1788,  1788,  1788,  ..., 28459, 28459, 28459],
        [16453,  4717,  2849,  ..., 18599, 11741,  6539]]), ('user', 'profile', 'keyword'): tensor([[27951, 27951, 27951,  ..., 28831, 28833, 28837],
        [  219,   224,   173,  ...,    25,    83,    71]]), ('user', 'post', 'tweet'): tensor([[     0,      0,      0,  ...,  28827,  28831,  28831],
        [     1,      7,      2,  ..., 118694, 118696, 118695]]), ('user', 'engage', 'tweet'): tensor([[ 3883,  3883,  2983,  ..., 28838, 28838, 28838],
        [25065, 25063, 22025,  ..., 32234, 75464, 75462]]), ('tweet', 'include', 'keyword'): tensor([[     0,      0,      0,  ..., 118696, 118696, 118696],
        [   215,     60,    173,  ...,    175,     80,    183]]), ('tweet', 'tag', 'keyword'): tensor([[    24,     24,     24,  ..., 118320, 118320, 1

In [ ]:
for num, edge in enumerate(edges):
  relation = edge['relation']
  print(relation)
  relation = relation.split('-')
  print(relation[1])
  relation = relation[1]
  print(relation)
  print('')
  break


user-follower-user
follower
follower



In [ ]:
data

HeteroData(
  user={
    x=[28839, 768],
    y=[28839],
  },
  tweet={ x=[118697, 768] },
  keyword={ x=[288, 768] },
  (user, to -->, user)={ edge_index=[2, 201142] },
  (user, to -->, keyword)={ edge_index=[2, 64997] },
  (keyword, <-- to, user)={ edge_index=[2, 64997] },
  (user, to -->, tweet)={ edge_index=[2, 162894] },
  (tweet, <-- to, user)={ edge_index=[2, 162894] },
  (tweet, to -->, keyword)={ edge_index=[2, 299997] },
  (keyword, <-- to, tweet)={ edge_index=[2, 299997] }
)

In [6]:
# Reaches around 91.8% Micro-F1 after 5 epochs.
import torch
import torch_geometric
from torch_geometric.nn import MetaPath2Vec

# metapath = [
#   ('user', 'follower', 'user'),
#   ('user', 'following', 'user'),
#   ('user', 'profile', 'keyword'),
#   ('user', 'post', 'tweet'),
#   ('user', 'engage', 'tweet'),
#   ('tweet', 'include', 'keyword'),
#   ('tweet', 'tag', 'keyword')
# ]
# # for i in range(len(metapath) - 1):
# #   if metapath[i][-1] != metapath[i + 1][0]:
# #     print(metapath[i][-1])
# #     print(metapath[i + 1][0])
# #     # raise ValueError(f"Invalid metapath: {metapath[i]} to {metapath[i + 1]}")
metapath = [
    ('user', 'to -->', 'user'),
        ('user', 'to -->', 'keyword'),
        ('keyword', '<-- to', 'user'),
        ('user', 'to -->', 'tweet'),
        ('tweet', '<-- to', 'user'),
        ('user', 'to -->', 'tweet'),
        ('tweet', 'to -->', 'keyword'),
        ('keyword', '<-- to', 'tweet'),
        ('tweet', '<-- to', 'user'),
]

# metapaths = [
#     ('user', 'to', 'user'),
#     ('user', 'to', 'keyword'),
#     ('keyword', 'to', 'user'),
#     ('user', 'to', 'tweet'),
#     ('tweet', 'to', 'user'),
#     ('user', 'to', 'tweet'),
#     ('tweet', 'to', 'keyword'),
#     ('keyword', 'to', 'tweet'),
#     ('tweet', 'to', 'user')
# ]

# metapath = [
#     ('user', 'post -->', 'tweet'),
#     ('tweet', '<-- engage', 'user'),
#     ('user', 'post -->', 'tweet'),
#     ('tweet', 'include -->', 'keyword'),
#     ('keyword', '<-- tag', 'tweet'),
#     ('tweet', '<-- post', 'user'),
#     ('user', 'profile -->', 'keyword'),
#     ('keyword', '<-- profile', 'user')
# ]
# metapath_updated = []
# for meta in metapath:
#   new_metapath = (meta[0], meta[1], meta[2])
#   metapath_updated.append(new_metapath)
# print(metapath_updated)

# metapath = [
#     ('user', 'to', 'user'),
#     ('user', 'to', 'tweet'),
#     ('tweet', 'to', 'keyword')
# ]


In [7]:
print(len(metapath))

9


In [ ]:
data

HeteroData(
  user={
    x=[28839, 768],
    y=[28839],
  },
  tweet={ x=[118697, 768] },
  keyword={ x=[288, 768] },
  (user, to -->, user)={ edge_index=[2, 201142] },
  (user, to -->, keyword)={ edge_index=[2, 64997] },
  (keyword, <-- to, user)={ edge_index=[2, 64997] },
  (user, to -->, tweet)={ edge_index=[2, 162894] },
  (tweet, <-- to, user)={ edge_index=[2, 162894] },
  (tweet, to -->, keyword)={ edge_index=[2, 299997] },
  (keyword, <-- to, tweet)={ edge_index=[2, 299997] }
)

In [8]:
print(os.getcwd())
# os.mkdir(os.path.join(os.getcwd(), 'models'))
os.path.exists(os.path.join(os.getcwd(), 'drive','MyDrive','Summer Enrichment program '))

print(os.path.exists('/content/drive/MyDrive/Summer Enrichment program '))

/content
True


In [ ]:
save_path = os.path.join(os.getcwd(), 'drive','MyDrive','Summer Enrichment program ' )
print(save_path)
torch.save(data, os.path.join(save_path, 'updated_twitter_hetero_data.pt'))

/content/drive/MyDrive/Summer Enrichment program 


In [ ]:
os.path.exists(os.path.join(os.path.join(os.getcwd(),'models'),'twitter_hetero_data.pt'))

False

In [ ]:
data = torch.load(os.path.join(os.getcwd(), 'drive','MyDrive','Summer Enrichment program ', 'twitter_hetero_data.pt'))

In [ ]:
data

HeteroData(
  user={
    x=[28839, 768],
    y=[28839],
  },
  tweet={ x=[118697, 768] },
  keyword={ x=[288, 768] },
  (user, to, user)={ edge_index=[2, 201142] },
  (user, to, keyword)={ edge_index=[2, 64997] },
  (user, to, tweet)={ edge_index=[2, 162894] },
  (tweet, to, keyword)={ edge_index=[2, 299997] }
)

In [9]:
data = torch.load(os.path.join(os.getcwd(), 'drive','MyDrive','Summer Enrichment program ', 'updated_twitter_hetero_data.pt'))

In [10]:
data

HeteroData(
  user={
    x=[28839, 768],
    y=[28839],
  },
  tweet={ x=[118697, 768] },
  keyword={ x=[288, 768] },
  (user, to -->, user)={ edge_index=[2, 201142] },
  (user, to -->, keyword)={ edge_index=[2, 64997] },
  (keyword, <-- to, user)={ edge_index=[2, 64997] },
  (user, to -->, tweet)={ edge_index=[2, 162894] },
  (tweet, <-- to, user)={ edge_index=[2, 162894] },
  (tweet, to -->, keyword)={ edge_index=[2, 299997] },
  (keyword, <-- to, tweet)={ edge_index=[2, 299997] }
)

In [11]:
import sklearn.metrics as metrics
import sklearn

In [12]:
print(metapath)

[('user', 'to -->', 'user'), ('user', 'to -->', 'keyword'), ('keyword', '<-- to', 'user'), ('user', 'to -->', 'tweet'), ('tweet', '<-- to', 'user'), ('user', 'to -->', 'tweet'), ('tweet', 'to -->', 'keyword'), ('keyword', '<-- to', 'tweet'), ('tweet', '<-- to', 'user')]


In [14]:
import torch
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('runing on gpu')
else:
    device = torch.device('cpu')

# model = MetaPath2Vec(data.edge_index_dict, embedding_dim=128,
#                      metapath=metapaths, walk_length=50, context_size=7,
#                      walks_per_node=5, num_negative_samples=5,
#                      sparse=True).to(device)
model = MetaPath2Vec(data.edge_index_dict, embedding_dim=128,
                     metapath=metapath, walk_length=3, context_size=4,
                     walks_per_node=5, num_negative_samples=5,
                     sparse=True).to(device)

loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

classifier = torch.nn.Linear(128, out_features=4).to(device)
classifier_optimizer = torch.optim.Adam(classifier.parameters(), lr=0.01)

def train(epoch, train_perm, valid_perm, test_perm, log_steps=200, eval_steps=1000):
    model.train()

    total_loss = 0
    for i, (pos_rw, neg_rw) in enumerate(loader):
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if (i + 1) % log_steps == 0:
            print(f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                  f'Loss: {total_loss / log_steps:.4f}')
            total_loss = 0

        if (i + 1) % eval_steps == 0:
            acc, f1_score = test()
            print(f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                  f'Acc: {acc:.4f}',
                  f'f1 score: {f1_score:.4f}')


    classifier_optimizer.zero_grad()

    y = data['user'].y
    user_embs = model('user')
    output = classifier(user_embs)
    # perm = torch.randperm(output.size(0))
    # train_perm = perm[:int(output.size(0) * 0.1)]
    # test_perm = perm[int(output.size(0) * 0.1):]
    # print(train_perm.shape, test_perm.shape)
    # logits = F.log_softmax(output, dim=)
    # pred_loss = F.nll_loss(output[train_perm], y[train_perm])
    _, predicted = torch.max(output[train_perm], dim = 1)
    acc_train = sklearn.metrics.accuracy_score(y[train_perm], output[train_perm] )
    pred_loss = F.cross_entropy(output[train_perm], y[train_perm])
    pred_loss.backward()
    classifier_optimizer.step()


    loss_val = F.cross_entropy(output[valid_perm], y[valid_perm])
    acc_val  = sklearn.metrics.accuracy_score(y[valid_perm], output[valid_perm])
    print('Train loss {:.4f}, Train Accuracy: {:.4f}'.format(pred_loss.item(), acc_train),
          'Val loss: {:.4f}, Val Accuracy: {:.4f}'.format(loss_val.item(), acc_val))

@torch.no_grad()
def test(test_perm, train_ratio=0.1):
    model.eval()

    # z = model('user', batch=data['user'].y_index.to(device))
    z = model("user") # 128
    # print(z.size())
    y = data['user'].y
    # print(y.size())

    # perm = torch.randperm(z.size(0))
    # train_perm = perm[:int(z.size(0) * train_ratio)]
    # test_perm = perm[int(z.size(0) * train_ratio):]
    # print(train_perm.shape, test_perm.shape)
    # TODO: Check the dimension of z


    # print(y[test_perm], z[test_perm]?)
    # z_prob = nn.Softmax(torch.softmax(z[test_perm], dim=1))
    _, predicted = torch.max(z[test_perm], dim =1)
    # _, predicted = torch.max(z_prob, dim =1)
    accuracy = sklearn.metrics.accuracy_score(y[test_perm], predicted)
    f1_score = sklearn.metrics.f1_score(y[test_perm], predicted, average="weighted")


    # return model.test(z[train_perm], y[train_perm], z[test_perm], y[test_perm],
                      # max_iter=150), metrics.f1_score(y[test_perm],
                      #                              z[test_perm].argmax(dim = 1),
                      #                              average='weighted')
    return accuracy, f1_score

for i in range(5):
  accuracy = []
  f1_socre = []
  perm = torch.randperm(28839)
  train_perm, valid_perm, test_perm = perm[:int(28839 * 0.1)], perm[int(28839 * 0.1):int(28839 * 0.2)], perm[int(28839 *0.2):]
  # print(train_perm.size(), valid_perm.size(), test_perm.size())
  print('Run: {}'.format(i + 1 ))
  for epoch in range(1, 20):
      train(epoch, train_perm, valid_perm, test_perm)
      acc, f1_score = test(test_perm)
      print(f'Epoch: {epoch}, Accuracy: {acc:.4f}',
            f'F1 score {f1_score:.4f}')
      accuracy.append(acc)
      f1_socre.append(f1_score)


Run: 1


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 1, Step: 00200/226, Loss: 9.2349


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [ ]:
import numpy as np

accuracy = np.array(accuracy)
f1_score = np.array(f1_socre)
print('Average Accuracy: {}'.format(np.mean(accuracy, axis= 0)))
print('Standard deviaition of Accuracy: {}'.format(np.std(accuracy, axis= 0)))
print('{:.3f} +/- {:.3f}'.format(np.mean(accuracy, axis= 0), np.std(accuracy, axis= 0)))
print('{:.3f} +/- {:.3f}'.format(np.mean(f1_score, axis=0), np.std(f1_score, axis=0)))




Average Accuracy: 0.006896286022499615
Standard deviaition of Accuracy: 0.0
0.007 +/- 0.000
0.013 +/- 0.000
